# `On the Incorporation of IPython’s Traitlets into Matplotlib`

Traitlets is a package used by IPython to simplify notification and configuration heuristics using pure Python. This is significant for Matplotlib because it provides a way to streamline the callback system, introduce basic type checking that will give useful tracback results, and improve the API by removing classic getter and setter functions in favor of setting or getting attributes directly from an object. On the other hand though, this is important for Traitlets as it's the first time it's been implamented in a large project outside of IPython. Thus, there's been back and forth communication in which changes in one project have influenced those being made in the other.

---

## `Matplotlib Specific Traitlets Tools`

Matplotlib presents specific use cases the standard Traitlets Package was not designed to handle, which means a few new tools needed to be introduced.

### `+ Notification Muting and Forced Callbacks`

Notification muting is used in several locations where updating an instance should alter its model, but not it's state. That is to say that values stored on the instance should be changed (i.e. updating the model), however notifications that would normally be fired as a result should be "muted" (i.e. state is maintained). Traitlets has a context manager for "holding" notifications, and while inside it, notifications are not triggered, but they are cached, and upon exiting, are fired. Thus, a context manager for "muting" has been included. The context manager itself is used explicitely only once or twice, however the method `private` is used extensively. It's found wherever private values on an instance (those named with a prefix of "`_`") were accessed or manipulated. The code below demonstrates that using `private` to set a value bypasses both validation and change notifiers, but still updates the value in `_trait_values`.

In [1]:
from matplotlib.traitlets import PrivateMethodMixin, Configurable, Int, validate, observe

class A(PrivateMethodMixin, Configurable):

        attr = Int(0)

        @validate('attr')
        def _attr_validate(self, commit):
            # should never be reached
            assert(False)

        @observe('attr')
        def _attr_changed(self, change):
            # should never be reached
            assert(False)

a = A()
# callbacks shouldn't be envoked
a.private('attr', 1)
# but the value of `attr` should be updated
assert(a.attr == 1)

In discussions with IPython devs who are working closely with traitlets, notification "muting" has been looked down upon. After all, the point of Traitlets is to trigger notifiers, not mute them. If it's possible, it might be adventageous to alter the functionality of matplotlib such that it wouldn't be needed. I have a feeling that might be a pretty substantial task though. One particularly difficult case comes to mind in which notification "holding" is required but notifications must be triggered midway through an inner context. The problem can be outlined as follows:

In [2]:
from traitlets import HasTraits, Int, observe

class Base(HasTraits):

    a = Int()
    b = Int()
    
    @validate('a')
    def _a_validator(self, commit):
        return commit['value']*2
    
    @observe('a')
    def _a_observer(self, change):
        self.b = change['new']

    def __init__(self):
        # do something with pre-notified state
        m = 'needs pre-notified state'
        assert self.a != self.b, m
        
        # post-notified state needs to be here
        
        # do something with post-notified state
        m = 'needs post-notified state'
        assert self.b == self.a, m

class Mixin(object):

    def __init__(self, value):
        with self.hold_trait_notifications():
            self.a = value
            super(Mixin, self).__init__()
        # post-notified state is actually here

def class_factory(klass):
    return type('FromFactory', (klass, Base), {})

inst = class_factory(Mixin)(1)

AssertionError: needs post-notified state

This only occured once during my refactor, but it does so within the contruction of `Subplot` classes which has relatively complex euristics associated with it. To solve the problem posed above I use two methods `mute_trait_notifications` which is a context mannager inspired by `hold_trait_notifications` that halts trait notifications, but still validates. Though notifications are muted, the manager yields a cache of all the changes that it captured before exiting. This makes it possible to force those changes later using `force_notify_changes` which simply uses `private` to set the value, and then calls `_notify_change` to call notifiers. Using these tools the solution, which is implamented similarly in the refactor, looks like this:

In [ ]:
from traitlets import HasTraits, Int, observe, validate
from matplotlib.traitlets import PrivateMethodMixin

class Base(PrivateMethodMixin, HasTraits):

    a = Int()
    b = Int()
    
    @validate('a')
    def _a_validator(self, commit):
        return commit['value']*2
    
    @observe('a')
    def _a_observer(self, change):
        self.b = change['new']

    def __init__(self, cached_changes):
        m = 'needs pre-notified state'
        assert self.a != self.b, m
        
        for name in cached_changes:
            # trigger notifications for the given changes
            self.force_notify_changes(*cached_changes[name])
            
        m = 'needs post-notified state'
        assert self.a == self.b, m

class Mixin(object):

    def __init__(self, value):
        with self.mute_trait_notifications() as cache:
            # the cache is dict keyed on trait names,
            # the values are lists of change dicts.
            self.a = value
            
        # values are validated
        assert self.a == value*2
        
        super(Mixin, self).__init__(cache)

def class_factory(klass):
    return type('FromFactory', (klass, Base), {})

inst = class_factory(Mixin)(1)

**\*note:** since `_notify_change` is planned to be made public (i.e. renamed to `notify_change`) the method `force_notify_change` which I've written, can probably be removed - it really only exists to wrap the privately defined `_notify_change`, and a user that needs to force notify should have enough knowledge to use the cache yielded by `mute_trait_notifications`.

### `+ Controling Values Upon Retrieval via Event Handlers`

When several attributes are requisite to compute the value of another, a high cost may be imposed to recomputing that value whenever one of its requirements changes. Additionally, if the attributes upon which the value depends exist on several instances (particularly if they're nested), setting up the change handlers that are needed to ensure that updates occur properly creates a lot of complexity. In either of these two cases, computing values at the time they are accessed in a localized function is preferable. For Matplotlib's transform types, though it's difficult to determine whether the former issue is significant enough to consider, the latter is definitely a concern. In the vanilla version of Matplotlib where classic getters and setters are used, getters for transform types returned a computed value that sometimes depended on several others. Now with Traitlets, a `RetrieveHandler` inheriting from `traitlets.EventHandler` is used to replicate this functionality inside a new `TraitType` called `TransformInstance`. A simple example showing how `@retrieve` (a decorator for the handler) can be used to compute values at the time they are accessed:

In [ ]:
from traitlets import HasTraits, Int
from matplotlib.traitlets import retrieve, OnGetMixin

# OnGetMixin will be explained below
class ActiveInt(OnGetMixin, Int):
    read_only=True

class A(HasTraits):
    
    i = ActiveInt(1)
    j = Int(1)
    k = Int(1)
    
    @retrieve('i')
    def _i_retriever(self, pull):
        # pull is a dictionary of the form:
        # {'value': the value that already exists
        #  'owner': the HasTraits instance value exists on,
        #  'trait': the trait which was triggered}
        return pull['value'] + self.j + self.k
    
a = A()
assert a.i == 3
a.j = 2
assert a.i == 4
a.k = 2
assert a.i == 5

The key piece to making `@retrieve` work is the `OnGetMixin` which overrides `__get__` to check if the a `RetrieveHandler` has registered a handler to its name. If it finds one, it will pass any value it can get from `TraitType.__get__` into the handler, and return the value given by the handler. Errors are raised when `TraitType.__get__` fails on a `TraitError` and a handler for the trait is absent. The `TraitType` for Matplotlib's transforms, `TransformInstance`, inherits from `OnGetMixin` along with `TraitType` and simply implaments custom validation logic.

#### `* Requirement Trees - Streamlining Simple Value Updates (an asside)`

If a value is accessed so many times that the cost of recomputing it outweighs the detriments of updating every time a requisite attribute changes, `@retrieve` shouldn't be used. As a side project, I worked on an `@requires` decorator which would solve this problem by linking up requisite attributes to a single callback in a relatively simple way. `@requires` was not used in the refactor or Matplotlib, nor was it discussed as a possible inclussion in Traitlets for the future, but it was fun to work on and has some interesting behaviors that are at least somewhat related to the work reported on here. The following example demonstrates some, but not all of what it can do, as showing everything would take more effort to explain than is worth doing here.

In [ ]:
# uses: https://github.com/rmorshea/antipackage
# not tested with: https://github.com/ellisonbg/antipackage
import antipackage as apkg
from github.rmorshea.misc.requirements import requires
from traitlets import HasTraits, Int

class A(HasTraits):
    a = Int()
    b = Int()
    
    # setup a requirement for an attribute that
    # exists on the same instance as `a`
    @requires('a', needs=('b',))
    def func(self, changes, values, handler):
        """The returned value is set to `a` when its requirements change
        
        changes: dictionary keyed on trait names whose values are change dicts
        values: dictionary keyed on trait names with the current values of those traits
        handler: the `RequiresHandler` instance
        """
        # if the value of the dynamically assigned
        # requirement `c` changes, update the value
        # of `b` on this instance to match it.
        if 'c' in changes:
            self.b += changes['c']['new']
            
        # a nested notification hold ensures that
        # the change to `b` will trigger after the
        # first call to func finishes. Thus, the
        # value of `a` is 2 as might intuitively
        # (but not technically) be expected.
        print(changes)
        print(values)
        
        total = sum(values.values())
        return total
    
    def __init__(self, other):
        # dynamically setup a requirement for
        # an attribute on another instance
        func = getattr(self.__class__, 'func')
        func.setup_requirement(other, 'c')
        
class B(HasTraits):
    
    c = Int()
    
b = B()
a = A(b)

b.c = 1

assert a.a == 2

### `+ New TraitTypes`

While only a handful of new `TraitTypes` have been required to implament traitlets within the base `Artist` class within matplotlib, there are plenty of opertunities to see them used constructively. Of those used within this refactor, the `TransformInstance` type mentioned above in relation to the computation of value upon retrieval is most significant. However it's purpose is reasonable subtle, a more obvious use for Traitlet's `TraitType` can be seen in the as yet, unused `Color` type. It accepts rgb, rgba, hex, or named colors, and depending on whether the `Color` instance was tagged with \``as_hex=True`\` or \``as_rgb=True`\` metadata, will coerce values. The examples below show off how `Color` might be used:

In [ ]:
from matplotlib.traitlets import Color
from traitlets import HasTraits

# test colors
black_values = ['#000000', '#000', (0, 0, 0, 255), 0, 0.0, (.0, .0, .0), (.0, .0, .0, 1.0)]
colored_values = ['#BE3537', (190, 53, 55), (0.7451, 0.20784, 0.21569)]

Test that a variety of color types can be coerced to hex strings properly.

In [ ]:
class A(HasTraits):
    color = Color().tag(as_hex=True)

a = A()

for values in black_values:
    a.color = values
    assert a.color == '#000000'

for values in colored_values:
    a.color = values
    
    assert a.color == '#be3537'

Test that a variety of color types can be coerced to rgb properly. Note that \``as_rgb=True`\` causes alpha values to be ignored.

In [ ]:
class A(HasTraits):
    color = Color().tag(as_rgb=True)

a = A()

for values in black_values:
    a.color = values
    assert a.color == (0.0, 0.0, 0.0)

for values in colored_values:
    a.color = values
    assert a.color == (0.7451, 0.20784, 0.21569)

Test named colors. The mapping that links names to hex colors comes from matplotlib.colors.cname

In [ ]:
from matplotlib.colors import cnames
class A(HasTraits):
    color = Color()

a = A()

# run through cnames to verify that
# they're all valid hex strings
for colorname in cnames.keys():
    a.color = colorname
    a.color = cnames[colorname]

Coercing values can involve some pretty nasty heuristic to get right. The color `TraitType` doesn't make them any simpler, but it does mean that what's required can be put out of sight and out of mind. The average user, even when investigating the source code at a surface level, won't have to dig through layers of logic to get at what they need, because the hard work is done behind the scenes rather than up front.

#### `* Dynamic Documentation of Traits on an Instance (an asside)`

Though `matplotlib.traitlets.Color` demonstrates that nasty coersion heuristics can be hidden from the average user some behaviors won't be intuitive to certain users despite our best efforts to minimize confusion. At a very basic level, someone unfamiliar with property-like attributes, or the Traitlets in general, may not expect coersion, or know that an attribute has notifiers associated with it, thus thorough documentation is essential. To combat this, a first attempt at dynamically generating a docstring in `__init__` that describes each `TraitType` on a `HasTraits` instance and its static notifiers was made in PR [`#131`](https://github.com/ipython/traitlets/pull/131) against the Traitlets repository. Since all the examples here are meant to be run with the [`master`](https://github.com/ipython/traitlets) branch, what's shown below is the expected output of a use case layed out in [`#131`](https://github.com/ipython/traitlets/pull/131):

```python
class A(HasTraits):
    i = Int().tag(allow_none=True, default_value=1)
    j = Any().tag(help='some help text for j', data={'x':1})

    def __init__(self, *args, **kwargs):
        """The constructor for the class
        
        More details about the constructor"""
        pass
    
    @default('j')
    def h(self): pass
    
    @validate('j')
    def g(self, commit):
        return commit['value']
    
    @observe('i', 'j')
    def f(self, change):
        pass
    
help(A().__init__)
```

If we are to apply the docstring of `A()` to the case of `matplotlib.traitlets.Color`, a description of the coersion characteristic for the `TraitType` would best noted through default help text that's stored in its metadata. The docstring isn't quite perfect though. The normal post processing that removes excess tabs that arise in multiline docstrings through Python's use of white space isn't working properly.

```
The constructor for the class

        More details about the constructor

:Traits of :class:`A` Instances:
        i : an int
            - trait metadata
                * ``allow_none = True``
                * ``default_value = 1``
            - event handlers
                * :method:`A.f` - observes changes

        j : any value
            :help: some help text for j

            - trait metadata
                * ``data = {'x': 1}``
            - event handlers
                * :method:`A.f` - observes changes
                * :method:`A.g` - validates values
                * :method:`A.h` - sets the default
```

## `Contributions To Traitlets`

As was mentioned at the begining of this report, the refactor of matplotlib has had some influence on the direction which Traitlets' developement has taken. Most of this influence has come in relation to the addition of various decorators meant to make improve the usability of Traitlets' callback system.

### `+ The @observe Decorator:`

Sylvain Corlay proposed the decorator and "change dicts" as an alternative to magic methods and lenghty function signatures part way through the refactor process. The decorator uses features of `traitlets.MetaHasTraits` to register notifiers to a `HasTraits` instance without relying on special naming practices like `_<trait_name>_changed`, to identify, and then trigger these functions later. Additionally change dicts (essentially dictionaries filled with key word arguments) make it easy to maintain backward compatability if new key word arguments are needed later. Now a callback function has a consistant signature that accepts one argument (the change dict), whereas before they could take between 1 and 4.

`@observe` hasn't changed much since Sylvain proposed it. About the only real change is the added ability to reference the function as a bound or unbound method depending on whether you accessed the `ObserveHandler` through the class or through the instance. It remains important though because it's the inspiration for `@validate` and `@default` which came late, and ultimately Matplotlib's `@retrieve`. The difference between the use of bound vs unbound references is shown below:

In [ ]:
from traitlets import HasTraits, Int, observe

class A(HasTraits):
    
    i = Int()
    j = Int()
    
    @observe('i')
    def _i_observe(self, change):
        self.j = change['new']
        
class B(A):
    
    @observe('i')
    def _i_observe(self, change):
        change['new'] = 2*change['new']
        # a reference to the bound method
        super(B, self)._i_observe(change)
        
class C(A):
    
    @observe('i')
    def _i_observe(self, change):
        change['new'] = 2*change['new']
        # a reference to the unbound method
        A._i_observe(self, change)
        
b = B()
b.i = 1
assert b.j == b.i*2

c = C()
c.i = 1
assert c.j == c.i*2

Since the refactor of Matplotlib was meant to work with the most recent version of Traitlets, once `@observe` was merged, Matplotlib was changed from relying on Traitlets' magic methods to using the new decorator api. Almost immediately though, it was apparent that the contrast between the magic methods still being used to perform cross-validation and assign default values dynamically and the new decorator was stark. Had `@observe` remained the only decorator in use, it may have added more confusion than it dispelled. To remedy this, the afformetioned decorators were quickly developed.

### `+ The @validate Decorator:`